In [1]:
from typing import List, Dict
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.pyplot as plt
import contextily as ctx
import geoviews as gv
import cartopy.crs as ccrs

import plotly.express as px

from dataclasses import dataclass

from sqlalchemy import create_engine, text
import psycopg2

# -------------------------------------
# Database Configuration (Postgres)
# -------------------------------------
db_config = {
    "host": "143.248.230.55",
    "port": "5432",
    "dbname": "AIS_DB_Prototype",
    "user": "root",
    "password": "password",
}

# Build SQLAlchemy engine
database_url = (
    f"postgresql://{db_config['user']}:{db_config['password']}"
    f"@{db_config['host']}:{db_config['port']}/{db_config['dbname']}"
)
engine = create_engine(database_url)

In [ ]:
sql = "select * from public.ais_korea limit 10"
res = pd.read_sql(sql, engine)

In [5]:
res.columns

Index(['mmsi', 'eta', 'longitude', 'latitude', 'sog', 'cog', 'heading',
       'destination', 'navstatus', 'posutc', 'posutcmin', 'geom', 'shiptype',
       'shiptypecode', 'shipclass'],
      dtype='object')

In [9]:
sql = """SELECT 
    COUNT(*) AS total_records,
    MIN(posutc) AS earliest_report,
    MAX(posutc) AS latest_report
FROM  public.ais_korea"""
res2 = pd.read_sql(sql, engine)
res2

,total_records,earliest_report,latest_report
0,36000000,2023-11-01 00:00:05,2024-03-31 23:59:49


In [10]:
sql = """SELECT 
    shiptype,
    shiptypecode,
    COUNT(*) AS vessel_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS percentage
FROM  public.ais_korea
WHERE shiptype IS NOT NULL
GROUP BY shiptype, shiptypecode
ORDER BY vessel_count DESC
LIMIT 10;"""
res3 = pd.read_sql(sql, engine)
res3

,shiptype,shiptypecode,vessel_count,percentage
0,GENERAL_CARGO,70,6473932,17.98
1,DRY_BULK,70,6116656,16.99
2,TANKER_PRODUCT,80,3761722,10.45
3,CONTAINER,70,3515786,9.77
4,TANKER_CHEMICALS,80,3401070,9.45
5,TUG,52,2755573,7.65
6,FISHING,30,2692428,7.48
7,GENERAL_TANKER,80,1219570,3.39
8,GAS_CARRIER,70,988491,2.75
9,TANKER_CRUDE,80,852519,2.37


In [11]:
sql = """SELECT 
    destination,
    COUNT(*) AS destination_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS percentage
FROM  public.ais_korea
WHERE destination IS NOT NULL AND destination != ''
GROUP BY destination
ORDER BY destination_count DESC
LIMIT 10;"""
res4 = pd.read_sql(sql, engine)
res4

,destination,destination_count,percentage
0,BUSAN,1955540,5.43
1,KR USN,1530042,4.25
2,KR PUS,1258343,3.50
3,ULSAN,1178418,3.27
4,Unknown,921105,2.56
5,KR YOS,888629,2.47
6,KRPUS,766080,2.13
7,YEOSU,641567,1.78
8,KR PTK,626702,1.74
9,KR INC,591379,1.64


In [12]:
sql = """SELECT 
    t.shiptype,
    t.shiptypecode,
    COUNT(DISTINCT t.mmsi) AS vessel_count,
    ROUND(COUNT(DISTINCT t.mmsi) * 100.0 / SUM(COUNT(DISTINCT t.mmsi)) OVER (), 2) AS percentage
FROM public.ais_korea t
WHERE t.shiptype IS NOT NULL
GROUP BY t.shiptype, t.shiptypecode
ORDER BY vessel_count DESC
LIMIT 10;"""
res5 = pd.read_sql(sql, engine)
res5

,shiptype,shiptypecode,vessel_count,percentage
0,DRY_BULK,70,5869,27.66
1,GENERAL_CARGO,70,4498,21.20
2,CONTAINER,70,2196,10.35
3,FISHING,30,2168,10.22
4,TANKER_PRODUCT,80,1204,5.67
5,TANKER_CHEMICALS,80,916,4.32
6,TANKER_CRUDE,80,634,2.99
7,TUG,52,493,2.32
8,GENERAL_TANKER,80,481,2.27
9,CAR_CARRIER,70,435,2.05


In [14]:
sql = """WITH VesselPoints AS (
    SELECT 
        mmsi,
        shiptype,
        shiptypecode,
        COUNT(*) AS point_count
    FROM public.ais_korea
    WHERE shiptype IS NOT NULL
    GROUP BY mmsi, shiptype, shiptypecode
)
SELECT 
    shiptype,
    shiptypecode,
    COUNT(DISTINCT mmsi) AS vessel_count,
    ROUND(AVG(point_count), 2) AS avg_points_per_vessel,
    SUM(point_count) AS total_points
FROM VesselPoints
GROUP BY shiptype, shiptypecode
ORDER BY avg_points_per_vessel DESC;"""
res6 = pd.read_sql(sql, engine)
res6

,shiptype,shiptypecode,vessel_count,avg_points_per_vessel,total_points
0,ANTI_POLLUTION,54,13,6613.77,85979.0
1,TUG,52,493,5589.40,2755573.0
2,VEHICLE_PASSENGER,60,106,5401.17,572524.0
3,ROLL_ON_ROLL_OFF,70,96,4521.29,434044.0
4,PASSENGER,60,132,3892.19,513769.0
5,TANKER_CHEMICALS,80,916,3712.96,3401070.0
6,TANKER_PRODUCT,80,1204,3124.35,3761722.0
7,REEFER,70,153,3006.85,460048.0
8,GENERAL_TANKER,80,481,2535.49,1219570.0
9,PILOT_VESSEL,50,14,2450.07,34301.0


## Functions

In [3]:
def load_fishing_vessels_voyages_with_loitering(sample_n: int = 5) -> pd.DataFrame:

    sql = text(
        """
        SELECT
            mmsi,
            CAST(eta AS TIMESTAMP)
            AS eta_time,
            COUNT(*)
            AS entry_count
            FROM public.loitering_new_v2
            GROUP BY
            mmsi,
            CAST(eta AS TIMESTAMP)
            HAVING
            COUNT(*) > 10
            ORDER BY
            entry_count desc,
            mmsi,
            eta_time
        LIMIT :n;
    """
    )
    return pd.read_sql(sql, engine, params={"n": sample_n}, parse_dates=["eta_time"])

def load_loitering_part_of_trajectory(mmsi: int, eta_time: pd.Timestamp) -> pd.DataFrame:
    sql = text(
        """
        SELECT 
            posutc AS ts_string,
            latitude,
            longitude,
            sog,
            cog,
            heading
        FROM public.loitering_new_v2
        WHERE mmsi = :m
          AND CAST(eta AS TIMESTAMP) = :e
        ORDER BY posutc::timestamp;
    """
    )
    df = pd.read_sql(sql, engine, params={"m": mmsi, "e": eta_time})
    df["timestamp"] = pd.to_datetime(df["ts_string"], errors="coerce")

    # Convert columns to numeric, coercing invalid values to NaN
    numeric_columns = ["latitude", "longitude", "sog", "cog", "heading"]
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df.drop(columns=["ts_string"], inplace=True)
    # Drop rows where any of the specified columns or timestamp is NaN
    return df.dropna(subset=["timestamp"] + numeric_columns)

def load_full_trajectory(mmsi: int, eta_time: pd.Timestamp) -> pd.DataFrame:
    """
    Load ALL AIS points (posutc, latitude, longitude, sog, cog, heading)
    for a given (mmsi, eta) from ais_korea.
    Returns a DataFrame with parsed timestamps.
    """
    sql = text(
        """
        SELECT 
            posutc AS ts_string,
            latitude,
            longitude,
            sog,
            cog,
            heading
        FROM public.ais_korea
        WHERE mmsi = :m
          AND CAST(eta AS TIMESTAMP) = :e
        ORDER BY posutc::timestamp;
    """
    )
    df = pd.read_sql(sql, engine, params={"m": mmsi, "e": eta_time})
    df["timestamp"] = pd.to_datetime(df["ts_string"], errors="coerce")

    # Convert columns to numeric, coercing invalid values to NaN
    numeric_columns = ["latitude", "longitude", "sog", "cog", "heading"]
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df.drop(columns=["ts_string"], inplace=True)
    # Drop rows where any of the specified columns or timestamp is NaN
    return df.dropna(subset=["timestamp"] + numeric_columns)


# latitude	longitude	sog	cog	heading	timestamp
@dataclass
class AISColumnNames:
    # Date: str = "Date"
    # Sampled_Date: str = "Sampled_Date"
    # Latitude: str = "Latitude"
    # Longitude: str = "Longitude"
    # Pseudo_Longitude: str = "Pseudo_Longitude"
    # SOG: str = "SOG"
    # COG: str = "COG"
    # Heading: str = "Heading"

    # n_Latitude: str = "norm Latitude"
    # n_Longitude: str = "norm Longitude"
    # n_SOG: str = "norm SOG"
    # n_COG: str = "norm COG"
    # n_Heading: str = "norm Heading"
    Date: str = "timestamp"
    Sampled_Date: str = "sampled_timestamp"
    Latitude: str = "latitude"
    Longitude: str = "longitude"
    Pseudo_Longitude: str = "pseudo_longitude"
    SOG: str = "sog"
    COG: str = "cog"
    Heading: str = "heading"

    n_Latitude: str = "norm_latitude"
    n_Longitude: str = "norm_longitude"
    n_SOG: str = "norm_sog"
    n_COG: str = "norm_cog"
    n_Heading: str = "norm_heading"

    is_synthetic: str = "is_synthetic"
    to_predict: str = "to_predict"


cols: AISColumnNames = AISColumnNames()
target_freq_in_minutes = 10
target_freq: str = f"{target_freq_in_minutes}min"
sample_T: pd.Timedelta = pd.Timedelta(minutes=target_freq_in_minutes)


def get_sampled_trajectory(trajectory: pd.DataFrame) -> pd.DataFrame:
    trajectory[cols.Date] = pd.to_datetime(trajectory[cols.Date])
    trajectory = trajectory.set_index(cols.Date)
    trajectory = trajectory.sort_index()

    # add first and last steps of trajectory which are divisible by 10 minutes
    first = trajectory.iloc[:1].copy()
    first.index = [trajectory.index.min().floor(target_freq)]
    last = trajectory.iloc[-1:].copy()
    last.index = [trajectory.index.max().ceil(target_freq)]
    trajectory = pd.concat([first, trajectory, last])

    # Define exact 10-minute sampling times
    start_time = trajectory.index.min().floor("h")  # Round down to the nearest hour
    end_time = trajectory.index.max().ceil("h")  # Round up to the nearest hour
    sampling_times = pd.date_range(start_time, end_time, freq=target_freq)

    # Filter only timestamps where at least one real record exists within ±10 minutes
    valid_sampling_times = [
        t for t in sampling_times if any(abs(trajectory.index - t) <= sample_T)
    ]

    trajectory = trajectory[~trajectory.index.duplicated(keep="first")]
    trajectory_interpolated = trajectory.reindex(
        trajectory.index.union(valid_sampling_times)
    ).sort_index()

    # Perform linear interpolation
    trajectory_interpolated = trajectory_interpolated.interpolate(method="time")

    # Keep only the sampled timestamps and drop any remaining NaNs
    trajectory_sampled = (
        trajectory_interpolated.loc[valid_sampling_times].dropna().reset_index()
    )
    trajectory_sampled.rename(columns={"index": cols.Sampled_Date}, inplace=True)
    return trajectory_sampled


def get_trajectory_sequences(
    trajectory_sampled: pd.DataFrame, time_column_name=None
) -> List[pd.DataFrame]:
    if time_column_name is None:
        time_column_name = cols.Sampled_Date
    trajectory_sequences: List[pd.DataFrame] = []  # To store the sequences
    current_sequence = pd.DataFrame(
        columns=trajectory_sampled.columns
    )  # DF To track the current sequence

    # Iterate through the timestamps
    for i in range(len(trajectory_sampled) - 1):
        if (
            trajectory_sampled[time_column_name][i + 1]
            - trajectory_sampled[time_column_name][i]
            == sample_T
        ):
            # If the difference is 10 minutes, add the current timestamp to the sequence
            if len(current_sequence) == 0:
                current_sequence = trajectory_sampled.iloc[
                    [i]
                ]  # Add the first timestamp of the sequence
            current_sequence = pd.concat(
                [current_sequence, trajectory_sampled.iloc[[i + 1]]],
                ignore_index=True,
            )  # Add the next timestamp
        else:
            # If the difference is not 10 minutes, end the current sequence
            if len(current_sequence) != 0:
                trajectory_sequences.append(
                    current_sequence
                )  # Store the completed sequence
                current_sequence = pd.DataFrame(
                    columns=trajectory_sampled.columns
                )  # Reset the current sequence

    # Handle the last sequence if it ends at the last timestamp
    if len(current_sequence) != 0:
        trajectory_sequences.append(current_sequence)

    return trajectory_sequences


def plot_plotly_trajectory_groups(
    df_groups: List[List[pd.DataFrame]],
    group_names,
    color_sequence=None,
    line_width=2,
    marker_size=4,
):
    if not df_groups:
        raise ValueError("Empty list of DataFrame groups provided")

    if color_sequence is None:
        color_sequence = px.colors.qualitative.Plotly

    # Create empty figure with proper mapbox setup
    fig = px.scatter_mapbox(lat=[None], lon=[None]).update_layout(
        mapbox_style="open-street-map", mapbox_zoom=8, height=600
    )
    min_lat = 360
    max_lat = 0
    min_lon = 360
    max_lon = 0
    for group_id, df_group in enumerate(df_groups):
        group_color = color_sequence[group_id % len(color_sequence)]

        for segment_id, df in enumerate(df_group):
            if len(df) == 0:
                continue  # Skip empty dataframes

            # Add line trace for this segment
            fig.add_trace(
                px.line_mapbox(
                    df,
                    lat=cols.Latitude,
                    lon=cols.Longitude,
                    color_discrete_sequence=[group_color],
                )
                .data[0]
                .update(
                    mode="lines+markers",
                    line=dict(width=line_width),
                    marker=dict(size=marker_size),
                    name=f"{group_names[group_id]}",
                    showlegend=(segment_id == 0),  # Only show legend for first segment
                    legendgroup=f"{group_names[group_id]}",
                    hoverinfo="text",
                    customdata=df[[cols.Sampled_Date, cols.SOG, cols.COG]],
                    hovertemplate=(
                        "Latitude: %{lat}<br>"
                        "Longitude: %{lon}<br>"
                        "Date: %{customdata[0]}<br>"
                        "SOG: %{customdata[1]}<br>"
                        "COG: %{customdata[2]}<br>"
                    ),
                )
            )

            # min/max lat/lot
            min_lat = min(min_lat, df[cols.Latitude].min())
            max_lat = max(max_lat, df[cols.Latitude].max())
            min_lon = min(min_lon, df[cols.Longitude].min())
            max_lon = max(max_lon, df[cols.Longitude].max())

    fig.update_layout(
        margin={"r": 0, "t": 40, "l": 0, "b": 0},
        showlegend=True,
        legend_title_text="Trajectory Groups",
        title="Vessel Trajectory",
    )

    # Auto-zoom to the data
    if len(df_groups) > 0 and len(df_groups[0]) > 0:
        fig.update_mapboxes(
            center=dict(lat=(min_lat + max_lat) / 2, lon=(min_lon + max_lon) / 2)
        )

    return fig

## Code

In [8]:
voyages_df = load_fishing_vessels_voyages_with_loitering(sample_n=100000)
voyages_df  # display the sample selection

,mmsi,eta_time,entry_count
0,431000306,2024-01-12 08:30:00,15116
1,440152970,2023-08-13 12:00:00,10055
2,431022371,2023-09-28 08:00:00,5486
3,431002273,2024-01-26 18:30:00,5029
4,431000523,2024-03-08 08:30:00,3900
...,...,...,...
2661,563727000,2023-11-03 07:10:00,11
2662,636018238,2024-03-29 22:15:00,11
2663,636018725,2024-02-20 02:00:00,11
2664,636020743,2023-12-29 22:30:00,11


In [9]:
# voyages_df = load_fishing_vessels_voyages(sample_n=100)
# voyages_df  # display the sample selection
current_voyages_df = voyages_df[1000:]

for idx, row in current_voyages_df.iterrows():
    print(idx, row)
    try:
        m       = int(row["mmsi"])
        eta_val = row["eta_time"]
        
        # Load full trajectory points
        df = load_full_trajectory(m, eta_val)
        sampled_boat_trajectory = get_sampled_trajectory(df)
        sampled_boat_trajectory.to_csv(f"../../data/loitering_sampled/mmsi_{m}_eta_val_{eta_val}.csv")
        # sample_traj_sequences = get_trajectory_sequences(sampled_boat_trajectory)
        # fig = plot_plotly_trajectory_groups([sample_traj_sequences], group_names=["Initial trajectory"])
        # fig.write_image(f"../../results/loitering_sampled/png_len_{len(sampled_boat_trajectory)}_mmsi_{m}_eta_val_{eta_val}.png")
        # fig.write_html(f"../../results/loitering_sampled/html_len_{len(sampled_boat_trajectory)}_mmsi_{m}_eta_val_{eta_val}.html")
        
        
        df_loitering = load_loitering_part_of_trajectory(m, eta_val)
        sampled_boat_trajectory_loitering = get_sampled_trajectory(df_loitering)
        sampled_boat_trajectory_loitering.to_csv(f"../../data/loitering_sampled/mmsi_{m}_eta_val_{eta_val}_loitering.csv")
        # sample_traj_sequences_loitering = get_trajectory_sequences(sampled_boat_trajectory_loitering)
        # fig = plot_plotly_trajectory_groups([sample_traj_sequences_loitering], group_names=["Initial trajectory"])
        # fig.write_image(f"../../results/loitering_sampled/png_len_{len(sampled_boat_trajectory_loitering)}_mmsi_{m}_eta_val_{eta_val}_loitering.png")
        # fig.write_html(f"../../results/loitering_sampled/html_len_{len(sampled_boat_trajectory_loitering)}_mmsi_{m}_eta_val_{eta_val}_loitering.html")
    except Exception as e:
        print(f"Couldn't process {row}: e")

1000 mmsi                     352002796
eta_time       2024-02-14 15:00:00
entry_count                    103
Name: 1000, dtype: object
1001 mmsi                     412412406
eta_time       2024-01-01 00:00:00
entry_count                    103
Name: 1001, dtype: object
1002 mmsi                     412420269
eta_time       2023-12-10 12:00:00
entry_count                    103
Name: 1002, dtype: object
1003 mmsi                     431013557
eta_time       2023-12-15 18:00:00
entry_count                    103
Name: 1003, dtype: object
1004 mmsi                     431015414
eta_time       2024-03-18 11:45:00
entry_count                    103
Name: 1004, dtype: object
1005 mmsi                     440313240
eta_time       2023-11-18 11:00:00
entry_count                    103
Name: 1005, dtype: object
1006 mmsi                     636091959
eta_time       2024-02-28 16:00:00
entry_count                    103
Name: 1006, dtype: object
1007 mmsi                     412200557
eta_time

/var/folders/xl/2j0v3vnj3gbfd5vvjwwz9lr00000gn/T/ipykernel_37211/2739462945.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trajectory[cols.Date] = pd.to_datetime(trajectory[cols.Date])


1690 mmsi                     412331125
eta_time       2023-01-01 00:00:00
entry_count                     38
Name: 1690, dtype: object
1691 mmsi                     412333326
eta_time       2023-01-01 01:01:00
entry_count                     38
Name: 1691, dtype: object
1692 mmsi                     413563740
eta_time       2024-02-12 18:00:00
entry_count                     38
Name: 1692, dtype: object
1693 mmsi                     431275000
eta_time       2024-03-06 20:30:00
entry_count                     38
Name: 1693, dtype: object
1694 mmsi                     431400282
eta_time       2023-12-17 16:20:00
entry_count                     38
Name: 1694, dtype: object
1695 mmsi                     431400282
eta_time       2023-12-20 18:40:00
entry_count                     38
Name: 1695, dtype: object
1696 mmsi                     431400282
eta_time       2023-12-22 06:20:00
entry_count                     38
Name: 1696, dtype: object
1697 mmsi                     440077880
eta_time

/var/folders/xl/2j0v3vnj3gbfd5vvjwwz9lr00000gn/T/ipykernel_37211/2739462945.py:76: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["timestamp"] = pd.to_datetime(df["ts_string"], errors="coerce")


2335 mmsi                     431010889
eta_time       2023-11-14 11:15:00
entry_count                     17
Name: 2335, dtype: object
2336 mmsi                     431013876
eta_time       2023-11-07 13:25:00
entry_count                     17
Name: 2336, dtype: object
2337 mmsi                     431013876
eta_time       2023-11-28 11:45:00
entry_count                     17
Name: 2337, dtype: object
2338 mmsi                     431015341
eta_time       2023-11-04 18:00:00
entry_count                     17
Name: 2338, dtype: object
2339 mmsi                     431015341
eta_time       2023-11-11 23:00:00
entry_count                     17
Name: 2339, dtype: object
2340 mmsi                     431015341
eta_time       2023-11-22 18:00:00
entry_count                     17
Name: 2340, dtype: object
2341 mmsi                     431015341
eta_time       2023-11-25 23:00:00
entry_count                     17
Name: 2341, dtype: object
2342 mmsi                     431015341
eta_time